In [9]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

data = pd.read_csv('Data/dacy_sentiment.csv')

train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b', max_features=1000)
train_vectors = vectorizer.fit_transform(train_data['body'])
test_vectors = vectorizer.transform(test_data['body'])

clf = LogisticRegression(random_state=42)
clf.fit(train_vectors, train_data['sentiment'])

##new_review = data.loc[0, 'body']
new_review = "Dårligt virkelig dårligt"
new_review_vector = vectorizer.transform([new_review])
predicted_sentiment = clf.predict(new_review_vector)[0]

#Tjekker hvor mange af hver der
sentiment_counts = data['sentiment'].value_counts()
print(sentiment_counts)

if predicted_sentiment == 1:
    print("Positive sentiment")
elif predicted_sentiment == 0:
    print("Neutral sentiment")
else:
    print("Negative sentiment")

sentiment
positive    118
negative     76
neutral       6
Name: count, dtype: int64
Negative sentiment


In [3]:
data = pd.read_csv('Data/dacy_sentiment.csv')
data

,title,body,rating,sentiment,predicted,prob_pos,prob_neu,prob_neg
0,Der var service på i dag,"Der var service på i dag, det var dejligt. Så...",5,positive,positive,0.999,0.001,0.001
1,Et af verdens ringeste kurér firmaer,Hvorfor i alverden man vælger at drive et verd...,2,negative,neutral,0.003,0.994,0.003
2,Rigtig god information om levering😃,Rigtig god information om levering😃,5,positive,positive,0.927,0.072,0.001
3,Meget hjælpsom chauffør,Meget hjælpsom chauffør. Pakken kom til den af...,5,positive,positive,0.981,0.017,0.003
4,Pakken kom fuldstændig som forventet.,Pakken kom fuldstændig som forventet.,3,neutral,neutral,0.023,0.964,0.014
...,...,...,...,...,...,...,...,...
195,Ren og pæn og hurtig levering…,Ren og pæn og hurtig levering indstillet ved d...,5,positive,positive,0.998,0.001,0.002
196,Hurtig og god behandling,Intet at udsætte !,5,positive,negative,0.002,0.002,0.996
197,Dårligt firma,Fik en pakke afhentet 28/9.Nu er pakken forsvu...,1,negative,neutral,0.022,0.651,0.327
198,Hurtig levering,God forsendelse,5,positive,positive,0.995,0.005,0.001
